In [34]:
import pandas as pd
from utils import *
from mappings import *

In [35]:
df = read_dbf('../data/raw/exped.DBF')

In [36]:
df.head()

,expid,peakid,year,season,host,route1,route2,route3,route4,nation,...,accidents,achievment,agency,comrte,stdrte,primrte,primmem,primref,primid,chksum
0,ANN260101,ANN2,1960,1,1,NW Ridge-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2442047
1,ANN269301,ANN2,1969,3,1,NW Ridge-W Ridge,None,None,None,Yugoslavia,...,Draslar frostbitten hands and feet,None,None,None,None,False,False,None,None,2445501
2,ANN273101,ANN2,1973,1,1,W Ridge-N Face,None,None,None,Japan,...,None,None,None,None,None,False,False,None,None,2446797
3,ANN278301,ANN2,1978,3,1,N Face-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2448822
4,ANN279301,ANN2,1979,3,1,N Face-W Ridge,NW Ridge of A-IV,None,None,UK,...,None,None,None,None,None,False,False,None,None,2449204


### Primary key

In [37]:
df.expid = df.expid.str.cat(df.year.astype(str), sep='_')

In [38]:
df.head()

,expid,peakid,year,season,host,route1,route2,route3,route4,nation,...,accidents,achievment,agency,comrte,stdrte,primrte,primmem,primref,primid,chksum
0,ANN260101_1960,ANN2,1960,1,1,NW Ridge-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2442047
1,ANN269301_1969,ANN2,1969,3,1,NW Ridge-W Ridge,None,None,None,Yugoslavia,...,Draslar frostbitten hands and feet,None,None,None,None,False,False,None,None,2445501
2,ANN273101_1973,ANN2,1973,1,1,W Ridge-N Face,None,None,None,Japan,...,None,None,None,None,None,False,False,None,None,2446797
3,ANN278301_1978,ANN2,1978,3,1,N Face-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2448822
4,ANN279301_1979,ANN2,1979,3,1,N Face-W Ridge,NW Ridge of A-IV,None,None,UK,...,None,None,None,None,None,False,False,None,None,2449204


### Expedition Countries
#### Host

In [39]:
# replace documentation id mapping with custom mapping
df.host = apply_map(df.host, host_map)
df = update_country_list(df, 'host')

#### Nation

In [40]:
# explode slash-separated values into scalar names
df_nations = df[['expid', 'nation']]
df_nations.nation = df_nations.nation.str.split('/')
df_nations = df_nations.explode('nation', ignore_index=True)

In [41]:
# swap names with country ids
df_nations = update_country_list(df_nations, 'nation')

#### Countries

In [47]:
df_countries = df[['expid', 'countries']]
df_countries.countries = df_countries.countries.str.split(',')
df_countries = df_countries.explode('countries', ignore_index=True)